In [ ]:
import numpy as np

In [ ]:
#fully connected layer
class dense_layer():
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    # [ [y1 = x1w11 + x2w12+...+xiw1i + b1] 
    #   [y2 = x1w21 + x2w22+...+xiw2i + b2]
    #   [yj = x1wj1 + x2wj2+...+xiwji + bj] ]
    def forward(self, input):
        self.input = input
        output = np.dot(self.weights, self.input) + self.bias
        return output
    
    # dW = dE/dY * X^{-1}
    # dB = dE/dY
    # dX = W^{-1} * dE/dY
    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        bias_gradient = output_gradient
        input_gradient = np.dot(self.weights.T, output_gradient)
        #update the coefficient
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * bias_gradient
        return input_gradient

In [ ]:
#Activation Function
class tanh_layer():
    def __init__(self):
        self.activation = lambda x : np.tanh(x)
        self.activation_derivative = lambda x : 1 - np.tanh(x) ** 2
    # [ [y1 = f(x1)] 
    #   [y2 = f(x2)]
    #   [yj = f(xi)] ]
    def forward(self, input):
        self.input = input
        output = self.activation(self.input)
        return output
    
    # dxi = dE / dyi (multiply) df/dxi
    def backward(self, output_gradient,learning_rate):
        next_gradient = np.multiply(output_gradient, self.activation_derivative(self.input))
        return next_gradient

In [ ]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_derivative(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)


In [ ]:
X = np.array([ [[0],[0]], 
               [[0],[1]], 
               [[1],[0]], 
               [[1],[1]] ])
Y = np.array([[0], [1], [1], [0]])
network = [
    dense_layer(2,4),
    tanh_layer(),
    dense_layer(4, 1),
    tanh_layer()
]
epochs = 10000
learning_rate = 0.1
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

for e in range(epochs):
    for i in range(4):
        # forward
        output = predict(network, X[i])

        # backward
        grad = mse_derivative(Y[i], output)
        for layer in reversed(network):
            grad = layer.backward(grad,learning_rate)


        if e % 1000 == 0:
            loss = np.mean(np.power(Y[i] - output, 2))
            print(f'Epoch {e}, Loss: {loss}')

In [ ]:
# decision boundary plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
points = []
for x1 in np.linspace(0, 1, 20):
    for x2 in np.linspace(0, 1, 20):
        z = predict(network, [[x1], [x2]])
        points.append([x1, x2, z[0,0]])

points = np.array(points)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2], cmap="winter")
plt.show()
